# Inverted Pendulum Example

Let's see how to make this API work with the inverted pendulum task! This reinforcement learning API requires 3 things to be defined before we start running algorithms:

+ BlackBoxModel: defines the problem--see below for an example!
+ Policy: this is where your domain knowledge comes in--define action space and feature functions
+ Solver: This is where the API takes over and you just specify what you want to use

In [ ]:
include(joinpath("..","src","ReinforcementLearning.jl"))
using ReinforcementLearning

## Define Black Box Model Functions

The BlackBoxModel type requires the following things to be defined:
+ `model`: a generic type that holds all your model parameters for a specific instance of your problem
+ `init(model,rng)`: generate an initial state
+ `observe(model,rng,state,action=None)`: return an observation based on your state (and action--this isn't quite ironed out yet)
+ `next_state(model,rng,state,action)`: generate a next state given your state, action and problem parameterization
+ `reward(model,rng,state,action)`: generate a reward based on your state and action and problem parameterization
+ `isterminal(model,state,action)`: return a boolean of whether a state (and action) is terminal or not

In [ ]:
using PyPlot
using Interact
import Iterators.product

typealias State Tuple{Float64,Float64}
typealias Action Float64

type InvertedPendulumModel <: Model
    g::Float64
    m::Float64
    l::Float64
    M::Float64
    alpha::Float64
    dt::Float64
    function InvertedPendulumModel(;
                                    g::Float64=9.81,
                                    m::Float64=2.,
                                    M::Float64=8.,
                                    l::Float64=0.5,
                                    dt::Float64=0.1)
        self = new()
        self.g = g
        self.m = m
        self.l = l
        self.M = M
        self.m = m
        self.alpha = 1/(m+M)
        self.dt = dt
        return self
    end
end

In [ ]:
#init2(m::GridWorldModel,rng::AbstractRNG) = (rand(rng,1:m.W),rand(rng,1:m.H))
init(m::InvertedPendulumModel,rng::AbstractRNG) = ((rand(rng)-0.5)*0.1,(rand(rng)-0.5)*0.1)

isend(rng::AbstractRNG,m::InvertedPendulumModel,s::State,a::Action) = abs(s[1]) >= pi/2

reward(rng::AbstractRNG,m::InvertedPendulumModel,s::State,a::Action) = abs(s[1]) < pi/2 ? 0.: -1.

function dwdt(m::InvertedPendulumModel,th::Float64,w::Float64,u::Float64)
    num = m.g*sin(th)-m.alpha*m.m*m.l*(w^2)*sin(2*th)*0.5 - m.alpha*cos(th)*u
    den = (4/3)*m.l - m.alpha*m.l*(cos(th)^2)
    return num/den
end


function rk45(m::InvertedPendulumModel,s::State,a::Action)
    k1 = dwdt(m,s[1],s[2],a)
    #something...
end

function euler(m::InvertedPendulumModel,s::State,a::Action)
    alph = dwdt(m,s[1],s[2],a)
    w_ = s[2] + alph*m.dt
    th_ = s[1] + s[2]*m.dt + 0.5*alph*m.dt^2
    if th_ > pi
        th_ -= 2*pi
    elseif th_ < -pi
        th_ += 2*pi
    end
    return (th_,w_)
end

function next(rng::AbstractRNG,m::InvertedPendulumModel,s::State,a::Action)
    a_offset = 20*(rand(rng)-0.5)
    a_ = a + a_offset
    
    return euler(m,s,a_)
    #something..
end

Here we also implement some quality of life functions, such as an explicity one-hot feature function for each state-action pair, and a visualization function

In [ ]:
nb_th_bins = 20
nb_w_bins = 20

exemplars = collect(product([-pi/4;0.;pi/4],[-1.;0;1.]))

dist(a::State,b::State) = norm([a[1]-b[1];a[2]-b[2]],2)


function generate_joint_featurefunction(m::InvertedPendulumModel)
    nb_feat = nb_th_bins*nb_w_bins
  function feature_function(s::State)
        active_indices = [ReinforcementLearning.bin(s[1],-pi,pi,nb_th_bins)+nb_th_bins*(ReinforcementLearning.bin(s[2],-1,1,nb_w_bins)-1)]
    phi = sparsevec(active_indices,ones(length(active_indices)),nb_feat)
    return phi
  end
  return feature_function
end


function generate_disjoint_featurefunction(m::InvertedPendulumModel)
    nb_feat = nb_th_bins + nb_w_bins
    function feature_function(s::State)
        active_indices = [ReinforcementLearning.bin(s[1],-pi,pi,nb_th_bins);ReinforcementLearning.bin(s[2],-1,1,nb_w_bins)]
        return sparsevec(active_indices,ones(2),nb_feat)
    end
end

function visualize(m::InvertedPendulumModel,s::State,a::Action)
    #NOTE: th = 0 is upright
    th = s[1] + pi/2.
    #base grid
    w = 1.5*m.l
    fill([-w,w,w,-w],[-w,-w,w,w],color="#FFFFFF",edgecolor="#000000")
    #draw cart
    dx = 0.05*sign(a)
    h = 0.1
    l = 0.125
    fill([-l+dx;l+dx;l+dx;-l+dx],[-h;-h;h;h],color="#FF0000")
    #draw pole
    u = m.l*cos(th) #+ dx
    v = m.l*sin(th)
    arrow(dx,0,u,v,width=m.l/10,head_width=0.,head_length=0.,color="#00FF00")
    #add cart direction (force)
    if abs(dx) > 0
        arrow(dx,0.,dx,0.,width=h,head_width=1.75*h,head_length=abs(dx),color="#0000FF")
    end
    #add pole velocity 
    du = -s[2]*m.l*sin(th)/5.
    dv = s[2]*m.l*cos(th)/5.
    arrow(u,v,du,dv,width=m.l/10,head_width=m.l/5,head_length=m.l/5,color="#FF00FF")
end

function visualize(m::InvertedPendulumModel,S::Array{State,1},A::Array{Action,1})
  assert(length(S) == length(A))
  f = figure()
  @manipulate for i = 1:length(S); withfig(f) do
    visualize(m,S[i],A[i]) end
  end
end

In [ ]:
_A = Action[-50;0;50]

In [ ]:
m = InvertedPendulumModel()

We now define the BlackBoxModel type. Note that we do not include an observation function in the constructor--in this case, it uses a default identity observation model

In [ ]:
bbm = BlackBoxModel(m,init,next,reward,isend) 

## Setting Up the Policy

In general for a policy, we have to define an ActionSpace (which we require to be exactly or a subset of the true action space), and feature function, which maps the state into a vector.

Tile coding is provided (the API for tilecoding needs work, however) for a quick and dirty function approximator in the continuous domain. For concreteness/generality, we include a function `cast_mc_state`, which in the most general case, will convert whatever state representation you have into an array of numbers

In [ ]:
feature_function = generate_radial_basis(exemplars,1.,dist)
feature_function = generate_joint_featurefunction(m)#generate_featurefunction(m,_A)
#feature_function = generate_disjoint_featurefunction(m)
A = DiscreteActionSpace(_A)

In [ ]:
policy = EpsilonGreedyPolicy(feature_function,A,rng=MersenneTwister(3234),eps=0.1)

## Choose and Set up your Solver

Currently, the following solvers are supported:
+ Forgetful LSTD(\lambda) / LS-SARSA (untested)
+ SARSA(\lamda) 
+ Q(\lambda)
+ GQ(\lambda)
+ Double Q learning (untested)
+ Deterministic Policy Gradient (unimplemented)
+ (Natural) Actor-Critic (unimplemented
+ LSPI/Batch TD (untested)
+ True Online TD

We just ask that you know a-priori how big your feature vectors are to make initialization easy

For now, we're going to use Q-learning variants, since they work well for this task (the constructors for other solvers are left commented out for the reader to experiment with at their own leisure)

In [ ]:
#there might be a smart way to stick this into a constructor, but for now...
nb_features = length(ReinforcementLearning.expand(policy.exp,policy.feature_function(bbm.state),domain(A)[1]))

#updater = ForgetfulLSTDParam(nb_features,alpha=0.001/3)
#updater = SARSAParam(nb_features,lambda=0.7,init_method="unif_rand",trace_type="replacing")
#updater = TrueOnlineTDParam(nb_features,lambda=0.95,init_method="unif_rand")
updater = QParam(nb_features,A,lambda=0.95)
updater = GQParam(nb_features,A,b=1e-7)
#mem_size = 50
#updater = LSPIParam(nb_features,mem_size,del=0.01,discount=0.99)

## Actually set up the real solver

Some random cool things supported include:
+ minibatching
+ experience replay
+ adaptive learning rates, e.g.:
    * momentum
    * nesterov momentum
    * rmsprop
    * adagrad
    * adadelta
    * adam
+ simulated annealing (probably shouldn't support this)


In [ ]:
solver = Solver(updater,
                lr=0.01,
                nb_episodes=5000,
                nb_timesteps=3000,
                discount=0.99,
                annealer=NullAnnealer(),
                mb=NullMinibatcher(),
                er=NullExperienceReplayer(),
                display_interval=1)

In [ ]:
trained_policy = solve(solver,bbm,policy)

## Evaluate Policy
Basically just run a couple of simulations -- the simulator api is a subset of the stuff you see in solver

In [ ]:
sim = Simulator(discount=0.99,nb_sim=50,nb_timesteps=250,visualizer=visualize) #stuff...

In [ ]:
#returns average reward for now...
R_avg = simulate(sim,bbm,trained_policy)

In [ ]:
visualize(m,sim.hist.S,sim.hist.A)
#note: joint features gets 8.82
#note disjoint features gets: 

Note that we have to call visualize externally. Currently getting the visualization to work two or three function calls in isn't quite working.

In [ ]:
sim.hist.S[end]